In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,f1_score,average_precision_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
from tensorflow import keras
from tensorflow.keras import layers
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import pickle
from imblearn.over_sampling import SMOTE 

#### mof

In [2]:
# dataset with FRAX CRFs and GRS
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X1, Y1 = data.get('X', []).values, data.get('Y', []).values
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s1, y_train_s1 = sm.fit_resample(x_train1, y_train1)

In [3]:
# dataset with FRAX CRFs (no grs)
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X2, Y2 = data.get('X_nogrs', []).values, data.get('Y', []).values
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s2, y_train_s2 = sm.fit_resample(x_train2, y_train2)

In [4]:
# Model 4 (FRAX CRFs + GRS)
batch_size=614
dropout=0.08405376347880598
epochs=43
learning_rate=0.6341016716921827
neurons=2317

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=31, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s1, y_train_s1,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred1 = nn.predict(x_test1)
y_pred_value1 = np.where(y_pred1 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred1,axis=1) 
auc = metrics.roc_auc_score(y_test1, y_pred_value1)
prauc = average_precision_score(y_test1, yscore_raw)
f1=f1_score(y_test1, y_pred_value1, average="weighted")
print("Model 4: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test1, y_pred_value1))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 4:
ANN Accuracy : 0.6857419980601358
ANN AUC : 0.7291888195891768
ANN PRAUC : 0.5205898574052006
ANN F1 : 0.7669314690579995


In [5]:
# Model 3 (FRAX CRFs + GRS)
batch_size=600
dropout= 0.1
epochs= 60
learning_rate= 1
neurons= 10000

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=31, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s1, y_train_s1,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred1 = nn.predict(x_test1)
y_pred_value1 = np.where(y_pred1 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred1,axis=1)
auc = metrics.roc_auc_score(y_test1, y_pred_value1)
prauc = average_precision_score(y_test1, yscore_raw)
f1=f1_score(y_test1, y_pred_value1, average="weighted")
print("Model 3: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test1, y_pred_value1))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 3:
ANN Accuracy : 0.684384093113482
ANN AUC : 0.7270105831358098
ANN PRAUC : 0.5202730951963284
ANN F1 : 0.7653230299885692


In [6]:
# Model 2 (FRAX CRFs)
batch_size=727
dropout=0.010074801781291724
epochs=60
learning_rate=0.010630405728982002
neurons=7277

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=30, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s2, y_train_s2,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred2 = nn.predict(x_test2)
y_pred_value2 = np.where(y_pred2 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred2,axis=1) 
auc = metrics.roc_auc_score(y_test2, y_pred_value2)
prauc = average_precision_score(y_test2, yscore_raw)
f1=f1_score(y_test2, y_pred_value2, average="weighted")
print("Model 2: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test2, y_pred_value2))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 2:
ANN Accuracy : 0.637439379243453
ANN AUC : 0.7150673384399444
ANN PRAUC : 0.5199010742835821
ANN F1 : 0.7300519913858992


In [7]:
# Model 1 (FRAX CRFs)
batch_size=600
dropout=0.01
epochs=60
learning_rate= 0.1
neurons= 10000

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=30, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s2, y_train_s2,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred2 = nn.predict(x_test2)
y_pred_value2 = np.where(y_pred2 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred2,axis=1) 
auc = metrics.roc_auc_score(y_test2, y_pred_value2)
prauc = average_precision_score(y_test2, yscore_raw)
f1=f1_score(y_test2, y_pred_value2, average="weighted")
print("Model 1: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test2, y_pred_value2))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 1:
ANN Accuracy : 0.6042090537019305
ANN AUC : 0.7000519303790531
ANN PRAUC : 0.5196941300394732
ANN F1 : 0.7038744037284573


#### hip

In [8]:
# dataset with FRAX CRFs and GRS
with open('ready_whi_sp23_hip_py38', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X1, Y1 = data.get('X', []).values, data.get('Y', []).values
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s1, y_train_s1 = sm.fit_resample(x_train1, y_train1)

In [9]:
# dataset with FRAX CRF (no grs)
with open('ready_whi_sp23_hip_py38', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X2, Y2 = data.get('X_nogrs', []).values, data.get('Y', []).values
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s2, y_train_s2 = sm.fit_resample(x_train2, y_train2)

In [10]:
# Model 4 (FRAX CRFs + GRS)
batch_size=379
dropout=0.16094530144701638
epochs=56
learning_rate=0.42106984357346283
neurons=1016

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=31, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s1, y_train_s1,epochs=epochs,batch_size=batch_size,verbose=0)

y_pred1 = nn.predict(x_test1)
y_pred_value1 = np.where(y_pred1 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred1,axis=1)
auc = metrics.roc_auc_score(y_test1, y_pred_value1)
prauc = average_precision_score(y_test1, yscore_raw)
f1=f1_score(y_test1, y_pred_value1, average="weighted")
print("Model 4: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test1, y_pred_value1))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 4:
ANN Accuracy : 0.7593840931134824
ANN AUC : 0.7820105831358098
ANN PRAUC : 0.5581009345794392
ANN F1 : 0.8439230299885692


In [11]:
# Model 3 (FRAX CRFs + GRS)
batch_size=600 
dropout= 0.1 
epochs= 20
learning_rate= 1 
neurons= 1000 


def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=31, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s1, y_train_s1,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred1 = nn.predict(x_test1)
y_pred_value1 = np.where(y_pred1 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred1,axis=1)
auc = metrics.roc_auc_score(y_test1, y_pred_value1)
prauc = average_precision_score(y_test1, yscore_raw)
f1=f1_score(y_test1, y_pred_value1, average="weighted")
print("Model 3: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test1, y_pred_value1))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 3:
ANN Accuracy : 0.7414393792434532
ANN AUC : 0.7720673384399444
ANN PRAUC : 0.5577383177570093
ANN F1 : 0.8070519913858992


In [12]:
# Model 2 (FRAX CRFs)
batch_size=310
dropout=0.010245102343775694
epochs=77
learning_rate=0.01037651141259692
neurons=1249

def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=30, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s2, y_train_s2,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred2 = nn.predict(x_test2)
y_pred_value2 = np.where(y_pred2 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred2,axis=1)
auc = metrics.roc_auc_score(y_test2, y_pred_value2)
prauc = average_precision_score(y_test2, yscore_raw)
f1=f1_score(y_test2, y_pred_value2, average="weighted")
print("Model 2: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test2, y_pred_value2))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 2:
ANN Accuracy : 0.7457419980601357
ANN AUC : 0.7321888195891768
ANN PRAUC : 0.5579660922734851
ANN F1 : 0.8249314690579995


In [13]:
# Model 1 (FRAX CRFs)
batch_size=600 
dropout= 0.1 
epochs= 20
learning_rate= 1 
neurons= 1000 


def create_model(neurons,learning_rate, dropout,batch_size, epochs):
    model = Sequential()
    model.add(Dense(neurons, input_dim=30, kernel_initializer='normal', 
                    activation='sigmoid'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt=keras.optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

nn = create_model(neurons,learning_rate, dropout,batch_size, epochs)
nn.fit(x_train_s2, y_train_s2,epochs=epochs,batch_size=batch_size,verbose=0)
y_pred2 = nn.predict(x_test2)
y_pred_value2 = np.where(y_pred2 > 0.5, 1, 0)
yscore_raw=np.argmax(y_pred2,axis=1) 
auc = metrics.roc_auc_score(y_test2, y_pred_value2)
prauc = average_precision_score(y_test2, yscore_raw)
f1=f1_score(y_test2, y_pred_value2, average="weighted")
print("Model 1: ")
print("ANN Accuracy : ", metrics.accuracy_score(y_test2, y_pred_value2))
print("ANN AUC : ", auc)
print("ANN PRAUC : ", prauc)
print("ANN F1 : ", f1)

Model 1:
ANN Accuracy : 0.6927179758558819
ANN AUC : 0.7552820241441182
ANN PRAUC : 0.5576930200811755
ANN F1 : 0.8072162248643044
